#### 고급 최적화
* 1. 제안-평가-등록 패러다임: <br/>
`maximize`는 내부적으로 `suggest`, `probe` 및 `register` 메서드를 래핑해 한번에 처리되기 때문에 `최적화 루프에 더 많은 제어가 필요한 경우` 사용할 수 있음.

#### 라이브러리 불러오기

In [1]:
from bayes_opt import UtilityFunction
from bayes_opt import BayesianOptimization

In [2]:
SEED = 1

#### 제안-평가-등록 패러다임

##### 기본적인 흐름

* 1. 대상함수 지정

In [3]:
# 대상함수 정의
def black_box_function(x, y):
    return -x ** 2 - (y - 1) ** 2 + 1

* 2. 대상함수를 특정하지 않는 채, 옵티마이저 지정

In [4]:
# 대상함수의 평가가 최적화객체에 의해 수행되지 않음.
# 이 함수의 평가가 다른 기계에서도 수행되거나, 다른 언어로 작성되었을 수 있기 때문
# `probe` 또는 `maximize` 메서드를 호출하지 않는 한 최적화객체는 대상함수를 무시함

optimizer = BayesianOptimization(
    f=None,     # 실행할 대상 함수를 직접 지정하지 않음
    pbounds={'x': (-2, 2), 'y': (-3, 3)},
    verbose=2,
    random_state=SEED
)

* 3. 최적화에 필요한 탐색 및 활용전략 설정

In [5]:
# 최적화에 적합한 다양한 탐색 및 활용전략 설정
utility = UtilityFunction(
    kind='ucb', # 탐색과 활용 사이의 균형 유지, 불확실한 지점을 더 많이 탐색
    kappa=2.5,  # UCB전략에서 얼마의 불확실성을 고려할지 지정
                # 값이 높으면 탐색을, 값이 낮으면 활용을 강조함
    xi=0.0      # 탐색전략에서 사용되는 파라미터로 얼마나 큰 개선을 고려할지 조정
                # 값이 높으면 큰 개선을 값이 낮으면 작은 개선을 탐색함
)

* 4. 다음에 탐색할 매개변수 조합을 확인

In [6]:
# 옵티마이저는 언제는 suggest인자로 utility를 이용해,
# 옵티마이저가 다음으로 탐색할 매개변수 조합을 제안할 수 있음
next_point_to_probe = optimizer.suggest(utility)
print(f'Next point to probe is: {next_point_to_probe}')

Next point to probe is: {'x': -0.331911981189704, 'y': 1.3219469606529488}


* 5. 해당 시점에서 대상 함수 평가: 이 부분은 반복 실행됨

In [7]:
# 이렇게 제안된 지점에서 함수를 자유롭게 평가할 수 있음
target = black_box_function(**next_point_to_probe)
print(f'Found the target value to be: {target}')

Found the target value to be: 0.7861845912690542


* 6. 해당 시점의 관찰된 대상값을 옵티마이저에 등록

In [8]:
# 마지막으로 할 일은, 관찰된 대상값을 옵티마이저에 알리는 것(등록)임.
optimizer.register(
    params=next_point_to_probe,
    target=target
)

##### 최대화 루프
위 4~6 단계들을 반복함으로써 최대화 메서드의 내부를 구현

In [9]:
for _ in range(5):
    next_point = optimizer.suggest(utility)
    target = black_box_function(**next_point)
    optimizer.register(params=next_point, target=target)
    print(target, next_point)
    
print(optimizer.max)

-18.49187152919165 {'x': 1.8861546000771092, 'y': -2.9917780942581977}
0.7911494590443674 {'x': -0.31764604716962586, 'y': 1.3285597809731806}
-7.0 {'x': -2.0, 'y': 3.0}
-7.0 {'x': 2.0, 'y': 3.0}
-7.503866469950651 {'x': -2.0, 'y': -1.122231483592365}
{'target': 0.7911494590443674, 'params': {'x': -0.31764604716962586, 'y': 1.3285597809731806}}


#### 불연속 매개변수 처리

In [10]:
# 대상함수 생성
def func_with_discrete_params(x, y, d):
    # 'd'값의 불연속 값(정수) 여부 체크
    assert type(d) == int
    
    return ((x + y + d) // (1 + d)) / (1 + (x + y) ** 2)

def function_to_be_optimized(x, y, w):
    d = int(w)
    return func_with_discrete_params(x, y, d)

In [11]:
optimizer = BayesianOptimization(
    f=function_to_be_optimized,
    pbounds={'x': (-10, 10), 'y': (-10, 10), 'w': (0, 5)},
    verbose=2,
    random_state=SEED
)

In [12]:
optimizer.set_gp_params(alpha=1e-3)
optimizer.maximize()

|   iter    |  target   |     w     |     x     |     y     |
-------------------------------------------------------------
| 1         | -0.06199  | 2.085     | 4.406     | -9.998    |
| 2         | -0.0344   | 1.512     | -7.065    | -8.153    |
| 3         | -0.2177   | 0.9313    | -3.089    | -2.065    |
| 4         | 0.1865    | 2.694     | -1.616    | 3.704     |
| 5         | -0.2187   | 1.022     | 7.562     | -9.452    |
| 6         | 0.1868    | 2.533     | -1.728    | 3.815     |
| 7         | 0.05119   | 3.957     | -0.6151   | 6.785     |
| 8         | 0.1761    | 0.5799    | 1.181     | 4.054     |
| 9         | 0.04045   | 4.004     | 4.304     | 2.656     |
| 10        | 0.07509   | 0.0       | 4.843     | 7.759     |
| 11        | 0.3512    | 0.0       | -5.713    | 7.072     |
| 12        | -0.8068   | 0.0       | -9.09     | 8.6       |
| 13        | 0.3774    | 0.3974    | -4.19     | 6.263     |
| 14        | 0.157     | 0.0       | -3.587    | 8.534     |
| 15    

#### 기본 가우시안 프로세스 튜닝
* 베이지안 최적화 알고리즘은 관찰된 매개변수 조합과 관련된 목표값을 기반으로 `가우시안 프로세스 회귀를 수행하여 작동`함.

##### 가우시안 프로세스에 매개변수 전달
* 대상 함수에 따라 기본 가우시안 프로세스의 기본 매개변수를 변경하는 것이 유용할 수 있음

In [13]:
optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={'x': (-2, 2), 'y': (-3, 3)},
    verbose=2,
    random_state=SEED
)
# 옵티마이저가 사용할 가우시안 프로세스 모델 구성
# alpha: 불확실성 조정(높으면 불확실성이 높아지고, 낮으면 낮아짐(단, 낮으면 과적합우려있음))
# n_restarts_optimizer: 가우시안 프로세스 모델 사용시 초기 추정치 시도 횟 수
#   높으면 성능은 향상되지만 자원(시간 포함)은 많이 소요됨
optimizer.set_gp_params(alpha=1e-3, n_restarts_optimizer=5)

In [14]:
optimizer.maximize(init_points=1, n_iter=5)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 1         | 0.7862    | -0.3319   | 1.322     |
| 2         | -18.49    | 1.886     | -2.992    |
| 3         | 0.7911    | -0.3176   | 1.329     |
| 4         | -6.11     | -1.763    | 3.0       |
| 5         | -2.895    | 1.533     | 2.243     |
| 6         | -4.806    | -2.0      | -0.3439   |
